# AI Overlord
## Descent: Journeys in the Dark AI Uprising

In [1]:
############### IMPORTS ###############

import numpy as np
import pandas as pd
from descent_ai import *

############### FORMATTING ###############

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', 1000)


In [2]:
# import archetype data
ARCHETYPES = pd.read_csv('archetype_import.csv')
ARCHETYPES.T.fillna('')

,0,1,2,3,4,5,6,7,8,9,10,11,12
archetype,Berserker,Bombardier,Controller,Minion,Normal,Skirmisher,Sniper,Soldier,Stalker,Swarm,Tank,Thief,Undead
archetype_first,False,False,False,False,True,False,False,False,False,True,False,False,True
type_melee,True,,,True,True,True,,True,True,True,True,True,True
type_ranged,,True,,True,True,True,True,,True,,,True,True
type_magic,,True,True,True,True,True,True,,True,,,,True
type_boss,True,True,True,,True,True,True,True,True,,True,,True
target,closest,"max targets, min current HP","max targets, highest movement",closest,closest,closest engaged,max base HP,closest unengaged,min current HP,"engaged, closest",max base HP,most gold,closest
range_modifier,,"(0, 2)","(0, 2)",,,,"(2, 4)",,,,,,
move_modifier,,,,,,"(3, 3)","(0, 3)",,"(0, 3)",,,,
special,"gains ""berserk"" at half HP",blast +1,stun,"2x spawn, 1HP",,"+3 move, swarm command target","+2 range, swarm command target",form a line starting form first target,"starts with stealth, +5 power dice for attacks in stealth",swarm,+2 armor,"starts with stealth, stealth again at half HP",surges drain HP


### Notes
* Enemies in this AI version just use basic hero attacks and ignore overlord powers
* Enemies are able to battle, run, guard, etc., depending on their AI description / role

### Target / Default Target
* First check for ideal target within range = engaged
* If none, DEFAULT TARGET: Most damaged > lowest total HP > lowest defense > heros choice
* repeat with more distant ranges until a target is chosen

### Definitions
* Aim = reroll misses, surges = range
* Battle = make 2 attacks
* Berserk = if the monster has one or more wound tokens on it, it rolls all 5 power dice when attacking
* Dodge = reroll all non-misses on dice with miss icons
* Drain = each HP of drain is treated as a leech attack (-surges, heal monster)
* Fear = attacker must spend 1 surge for every Fear rank or the attack misses
* Flee = move max distance away from characters, maximize distance between monsters
* Form a line = 1st monster ends turn at range to target closest to obstruction/wall, subsequent enemies continue line to block maximum movement, starting at a wall when possible, prioritising the line over their preferred target when targets can be substituted
* Guard = makes an interrupt attack when a hero enters range
* Leech = for each wound lost due to a Leech attack, the target also loses 1 fatigue (or suffers 1 additional wound, ignoring armor, if out of fatigue) and the attacker is healed of 1 wound
* Range = the range at which enemies attacks are optimal (note: enemy orders include ±1 range when checking if "at range")
* Run = move up to 2x speed
* Stealth = like an invisibility potion, roll stealth die to attack, every turn roll power die - surge means no more stealth
* Swarm = a figure with Swarm may roll 1 extra power die for every other friendly figure adjacent to its target
* Swarm = attack the swarm target, attacks gain the swarm effect
* Undying n = Undying effect (roll power die, surge is ressurect) using n dice


In [3]:
class MonsterAI:
    # TODO: for now master and minions have the same archetype, it is possible to tweak this

    def __init__(self, monster_name, archetype=None, boss=False):
        if monster_name not in list(MONSTER_DF.name):
            raise ThisIsNotAMonsterYouCanChoose
        self.monster_name = monster_name
        self.boss = boss
        self._archetype_data = self.get_archetype_data(archetype)

    def get_archetype_data(self, archetype):
        columns = ['archetype', 'archetype_first', 'target', 'range_modifier', 'move_modifier', 'special', 'boss_special', 'lt_range', 'at_range', 'gt_range', 'gt_move_range']
        all_eligible_archetypes = ARCHETYPES.loc[ARCHETYPES[self.monster_name].notna(), columns]
        if archetype:
            if archetype not in list(all_eligible_archetypes['archetype']):
                raise ThisIsNotAReasonableChoiceForThatMonsterError
            return all_eligible_archetypes[all_eligible_archetypes['archetype']==archetype]
        return all_eligible_archetypes.sample()

    @property
    def archetype(self):
        return self._archetype_data['archetype'].iloc[0]

    @property
    def target(self):
        return self._archetype_data['target'].iloc[0]
    
    @property
    def range(self):
        range = MONSTER_DF.loc[MONSTER_DF['name']==self.monster_name, 'range'].iloc[0]
        if self._archetype_data['range_modifier'].notnull().iloc[0]:
            if self.boss:
                range += eval(self._archetype_data['range_modifier'].iloc[0])[1]
            else:
                range += eval(self._archetype_data['range_modifier'].iloc[0])[0]
        return range

    @property
    def movement(self):
        movement = MONSTER_DF.loc[MONSTER_DF['name']==self.monster_name, 'movement'].iloc[0]
        if self._archetype_data['move_modifier'].notnull().iloc[0]:
            if self.boss:
                movement += eval(self._archetype_data['move_modifier'].iloc[0])[1]
            else:
                movement += eval(self._archetype_data['move_modifier'].iloc[0])[0]
        return movement
    
    @property
    def lt_range(self):
        return self._archetype_data['lt_range'].iloc[0]
    
    @property
    def at_range(self):
        return self._archetype_data['at_range'].iloc[0]
    
    @property
    def gt_range(self):
        return self._archetype_data['gt_range'].iloc[0]
    
    @property
    def gt_move_range(self):
        return self._archetype_data['gt_move_range'].iloc[0]

    @property
    def special(self):
        if self.boss:
            return self._archetype_data['boss_special'].iloc[0]
        return self._archetype_data['special'].iloc[0]
    
    @property
    def archetype_first(self):
        return self._archetype_data['archetype_first'].iloc[0]

    def boss_stuff(self):
        # TODO: add random boss stuff (2x hitpoints, etc)?
        pass

    def summary(self):
        print('------------------------------------------------------------------')
        if self.archetype == 'Normal':
            print(self.monster_name)
        elif self.archetype_first:
            print(f"{self.archetype} {self.monster_name}")
        else:
            print(f"{self.monster_name} {self.archetype} ")
        print('------------------------------------------------------------------\n')

        if pd.notnull(self.special):
            print('Special Powers:', self.special.title().replace('Hp', 'HP'))

        print('Target:', self.target.title())

        attack_type = MONSTER_DF.loc[MONSTER_DF['name']==self.monster_name, 'attack_type'].iloc[0]
        if attack_type == 'melee':
            attack_range = 1 if 'reach' in MONSTER_DF.loc[MONSTER_DF.name==self.monster_name, 'abilities'].iloc[0].lower() else 0
            print('Attack Range:', attack_range, '\n')
        else:
            attack_range = self.range
            print('Attack Range:', attack_range-1, 'to', attack_range+1, '\n')

        print('Combat AI Instructions')
        print('----------------------')

        if attack_type == 'melee':
            if self.at_range:
                print('If at attack range: ', self.at_range)
            if self.gt_range:
                print(f'If within {self.movement + attack_range} spaces: ', self.gt_range)
            if self.gt_move_range:
                print(f'Otherwise: ', self.gt_move_range)
        else:
            if self.lt_range:
                print(f'If closer than {self.range-1} spaces: ', self.lt_range)
            if self.at_range:
                print(f'If {attack_range-1} to {attack_range+1} spaces: ', self.at_range)
            if self.gt_range:
                print(f'If within {self.movement + attack_range} spaces: ', self.gt_range)
            if self.gt_move_range:
                print('Otherwise: ', self.gt_move_range)
        
        print()



In [4]:
class AIOverlord:
    def __init__(self, monsters_name_list, boss_monster_name=None, all_undead=True, archetype=None):
        self.monsters = dict()
        self._archetype = archetype
        self._not_undead = False
        for monster in monsters_name_list:
            for _ in range(10):
                self.monsters[monster] = MonsterAI(monster, self._archetype)
                if self._not_undead and self.monsters[monster].archetype == 'Undead':
                    continue
                if all_undead and self.monsters[monster].archetype == 'Undead':
                    self._archetype = 'Undead'
                    break
                if all_undead and self.monsters[monster].archetype != 'Undead':
                    self._not_undead = True
                break
        self.boss = None if not boss_monster_name else MonsterAI(boss_monster_name, self._archetype)

    def summary(self, show_boss=True):
        print('***************************  MONSTERS  ***************************\n')
        for monster in self.monsters.values():
            monster.summary()
        if show_boss and self.boss:
            print('\n*****************************  BOSS  *****************************\n')
            self.boss.summary()
        print('******************************************************************')

    def boss_summary(self):
        if self.boss:
            print('\n*****************************  BOSS  *****************************\n')
            self.boss.summary()
            print('******************************************************************')
        else:
            print('There is no boss for this quest.')

class AIQuest:
    def __init__(self, n_monsters=3, n_encounters=4, boss=True, sorted_battles=True, use_all_minis=False, all_undead=True, archetype=None):
        self.monster_obj = Monsters(n_monsters, n_encounters, boss, sorted_battles, use_all_minis)
        self.monsters = self.monster_obj.quest_monsters
        self.boss = self.monster_obj.quest_boss
        self.overlord_obj = AIOverlord(self.monsters, self.boss, all_undead, archetype)

    def summary(self):
        self.encounter_summary()
        print()
        self.overlord_obj.summary()

    def monsters_summary(self, show_boss=False):
        self.overlord_obj.summary(show_boss)

    def boss_summary(self):
        self.overlord_obj.boss_summary()

    def encounter_summary(self):
        print('**************************  ENCOUNTERS  **************************\n')
        for encounter in self.monster_obj.encounters.values():
            print(encounter)


In [6]:
# TODO: update instructions for dragon and hellhound specifically because of breath weapon (or check for 'breath')

AIQuest().monsters_summary()

***************************  MONSTERS  ***************************

------------------------------------------------------------------
Dragon Bombardier 
------------------------------------------------------------------

Special Powers: Blast +1
Target: Max Targets, Min Current Hp
Attack Range: 2 to 4 

Combat AI Instructions
----------------------
If closer than 2 spaces:  move to range > attack
If 2 to 4 spaces:  aim > attack
If within 7 spaces:  move to range > attack
Otherwise:  stay move away from closest hero

------------------------------------------------------------------
Ferrox Skirmisher 
------------------------------------------------------------------

Special Powers: +3 Move, Swarm Command Target
Target: Closest Engaged
Attack Range: 0 

Combat AI Instructions
----------------------
If at attack range:  attack > flee
If within 7 spaces:  move to range > attack > flee
Otherwise:  move to range > attack (if possible)

-----------------------------------------------------